<a href="https://colab.research.google.com/github/ChangeIsKey/NovelSenseDiscovery/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ares
#!wget http://sensembert.org/resources/ares_embedding.tar.gz
#!tar -xf ares_embedding.tar.gz
!ls

In [ ]:
# sensebert sample
#!rm -r NovelSenseDiscovery
!git clone https://github.com/ChangeIsKey/NovelSenseDiscovery

#!cd NovelSenseDiscovery/data/ && tar -xf sensembert_data.tar.gz
#!ls NovelSenseDiscovery/data/sensembert_data 
#sense_vectors = 'NovelSenseDiscovery/data/sensembert_data/sensembert_EN_supervised.txt'

!cd NovelSenseDiscovery/data/ && tar -xf ares_embedding.tar.gz
!ls NovelSenseDiscovery/data/ares_embedding
#sense_vectors = 'NovelSenseDiscovery/data/ares_embedding/ares_bert_large_sample.txt'
sense_vectors = 'ares_embedding/ares_bert_large.txt'

In [ ]:
# get discovery repo
#!rm -r LSCDiscovery
!git clone https://github.com/seinan9/LSCDiscovery
!pip install -r LSCDiscovery/requirements.txt
!python -m spacy download en_core_web_sm
#!pip install --force-reinstall -v "spacy==3.1.1"

In [ ]:
#load vectors
#!git clone https://github.com/Garrafao/LSCDetection
'''
from LSCDiscovery.modules.utils_ import Space

# Load matrices
try: 
    space = Space(sense_vectors, format='npz') 
except ValueError: 
    space = Space(sense_vectors, format='w2v')

sense_vecs = space.matrix.toarray() #much more efficient
#split vectors
sense_vecs = sense_vecs[:,:1024] #use synset embedding
#sense_vecs = sense_vecs[:,1024:] #use sense embedding
row2id_senses = space.row2id
id2row_senses = space.id2row

#display(sense_vecs)
#display(row2id_senses)
#display(sense_vecs.shape)
'''

In [ ]:
import numpy
from gensim.models import KeyedVectors
sense_vecs = KeyedVectors.load_word2vec_format(sense_vectors, binary=False)
row2id_senses = {k:i for (i,k) in enumerate(sense_vecs.vocab.keys())}
id2row_senses = {i:k for (i,k) in enumerate(sense_vecs.vocab.keys())}
#display(id2row_senses)
vecs = []
for i in sense_vecs.vocab.keys():
    vecs.append(sense_vecs.get_vector(i))
sense_vecs = numpy.array(vecs) #much more efficient

#split vectors
sense_vecs = sense_vecs[:,:1024] #use synset embedding
#sense_vecs = sense_vecs[:,1024:] #use sense embedding

In [ ]:
# get benchmark
#!git clone https://github.com/ChangeIsKey/LSCDBenchmark
#!pip install -r LSCDBenchmark/requirements.txt
#!pip install torch

In [ ]:
#import sys 
#sys.path.append('./LSCDBenchmark/') 
#import main

In [ ]:
# download semeval corpora
#!cd LSCDiscovery && bash scripts/import_semeval_en.sh
#!ls data

In [ ]:
# sample uses
#!cd LSCDiscovery && bash -e scripts/prepare_sample.sh en_semeval sample_1 2 10 en

In [ ]:
#intersect sense and corpus vocab

In [ ]:
# extract usage vectors 
#!cd LSCDiscovery && bash scripts/discover_bert.sh en_semeval sample_1 en-large toklem 1+12 0.1

In [ ]:
#extract wsd data
!cd NovelSenseDiscovery/data/ && tar -xf data_files.tar.gz
#!ls NovelSenseDiscovery/data/data_files

In [ ]:
# visualize targets
#!ls LSCDiscovery/data/en_semeval/samples/sample_1
#targets_path = 'LSCDiscovery/data/en_semeval/samples/sample_1/sample.tsv'
targets_path = 'NovelSenseDiscovery/data/all.tsv'

import csv

with open(targets_path, encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile,fieldnames=['target'], delimiter='\t',quoting=csv.QUOTE_NONE,strict=True)
    targets = [row['target'] for row in reader]

#display(targets)

#uses_path = 'LSCDiscovery/data/en_semeval/samples/sample_1/usages_corpus1/casket.tsv'
#uses_path = 'NovelSenseDiscovery/data/data_files/1970s.tsv'
#!ls LSCDiscovery/data/en_semeval/samples/sample_1/usages_corpus1
# Uses
#with open(uses_path, encoding='utf-8') as csvfile:
#    reader = csv.DictReader(csvfile, delimiter='\t',quoting=csv.QUOTE_NONE,strict=True)
#    rows = [row for row in reader]
#display(rows)

In [ ]:
#vectorize wsd test usages
'''
%%bash
files=NovelSenseDiscovery/data/data_files
outputdir=NovelSenseDiscovery/data/data_files_vectors
rm -r $outputdir
mkdir $outputdir
cd LSCDiscovery
i=0
for file in ../$files/*.tsv
do
    echo $file
    python contextualized/bert.py -l $file ../$outputdir/$(basename "${file%.*}") en-large toklem 1+12
    if [ $i -eq 20 ];
    then
        break
    fi
    i=$(( $i + 1 ))
    #break
done
'''

In [ ]:
import sys 
sys.path.append('LSCDiscovery/modules/')
import csv
import gzip
import logging
import time
import numpy as np
from sklearn import preprocessing
import torch
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModel
from utils_ import Space

file_folder = 'NovelSenseDiscovery/data/data_files'
output_folder = 'NovelSenseDiscovery/data/data_files_vectors'
language = 'en-large'
type_sentences = 'toklem'
layers = '1+12'

is_len = True

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary) and model (weights)
model_language = {
        'en-large': 'bert-large-cased', 
        'en': 'bert-base-cased', 
        'de': 'bert-base-german-cased', 
        'sw': 'KB/bert-base-swedish-cased', 
        'multi': 'bert-base-multilingual-cased'
        }

tokenizer = BertTokenizer.from_pretrained(model_language[language])
model = BertModel.from_pretrained(model_language[language], output_hidden_states=True)

if type_sentences == 'toklem':
    type_ = 'token'
else:
    type_ = type_sentences

for target in targets[:20]:

    start_time = time.time()

    display(target)

    path_usages = file_folder+'/'+target+'.tsv'
    path_output = output_folder+'/'+target
    
    # Load sentences 
    context_vector_list = []
    test_sentences = []
    with open(path_usages, 'r') as csvFile:
        reader = csv.DictReader(csvFile, delimiter="\t", quoting=csv.QUOTE_NONE, strict=True)
        for row in reader:
            test_sentences.append(dict(row))

        # Create the vectors
        for i in range(0, len(test_sentences)):
            try:
                # Create target word(s)
                target_words = []
                target_word = str(test_sentences[i]["sentence_"+type_].split()[int([test_sentences[i]["index_"+type_]][0])])
                if type_sentences == 'toklem':
                    original_word = test_sentences[i]["lemma"]
                    target_words.append(tokenizer.tokenize(original_word))
                else:
                    clean_target_word = "".join(char for k,char in enumerate(target_word) if char.isalpha() or char == "-" or (char == "'" and k == len(target_word)-1))
                    if clean_target_word[-1] == "'":
                        clean_target_word = test_sentences[i]["lemma"]
                    target_words.append(tokenizer.tokenize(clean_target_word))
                target_words = target_words[0]
                
                # Tokenize text
                text = test_sentences[i]["sentence_"+type_]
                if type_sentences == 'toklem':
                    text = text.replace(target_word, original_word) 
                marked_text = "[CLS] " + text + " [SEP]"
                tokenized_text = tokenizer.tokenize(marked_text)
            
                # Search the indices of the tokenized target word in the tokenized text
                target_word_indices = []
                for j in range(0, len(tokenized_text)):
                    if tokenized_text[j] == target_words[0]:
                        for l in range(0, len(target_words)):
                            if tokenized_text[j+l] == target_words[l]:
                                target_word_indices.append(j+l)
                            if len(target_word_indices) == len(target_words):
                                break
                
                if len(target_word_indices) == 0:
                    logging.info("INDICES NOT FOUND. SKIPPED SENTENCE "+str(i))
                    continue

                # Trim tokenized_text if longer than 512
                if len(tokenized_text) > 512:
                    while (len(tokenized_text) > 512):
                        if tokenized_text[-1] != tokenized_text[target_word_indices[-1]]:
                            del(tokenized_text[-1])
                        else:
                            del(tokenized_text[0])
                            for index, value in enumerate(target_word_indices):
                                target_word_indices[index] -= 1
                
                # Create BERT Token Embeddings
                indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
                segments_ids = [1] * len(tokenized_text)
                tokens_tensor = torch.tensor([indexed_tokens])
                segments_tensors = torch.tensor([segments_ids])
                model.eval()
                with torch.no_grad():
                    outputs = model(tokens_tensor, segments_tensors)
                    hidden_states = outputs[2]
                token_embeddings = torch.stack(hidden_states, dim=0)
                token_embeddings = torch.squeeze(token_embeddings, dim=1)
                token_embeddings = token_embeddings.permute(1, 0, 2)
                vectors = []
                for number in target_word_indices:
                    token = token_embeddings[number]
                    layers_list = layers.split('+')
                    layers_list = list(map(int, layers_list))
                    #vec = [np.array(token[l]) for l in layers_list]
                    sum_vec = np.sum([np.array(token[l]) for l in layers_list], axis=0)
                    #sum_vec = np.sum([np.array(token[12]), np.array(token[1])], axis=0)
                    vectors.append(np.array(sum_vec))
                context_vector_list.append(np.sum(vectors, axis=0, dtype=float))
            except:
                print("SKIPPED SENTENCE "+str(i))
        
    
    # Normalize vectors in length
    if is_len:
        context_vector_list = preprocessing.normalize(context_vector_list, norm='l2')

    # Save contextVectorList_sparse matrix
    rows = [row['instance_id'] if 'instance_id' in row else str(i) for i, row in enumerate(test_sentences)]
    outSpace = Space(matrix=context_vector_list, rows=rows, columns=[])
    outSpace.save(path_output)

    logging.info("--- %s seconds ---" % (time.time() - start_time))
    print("")

In [ ]:
#load vectors for each target and assign nearest neighbor sense
#!ls LSCDiscovery/output/en_semeval/BERT_layers1+12_typetoklem/discovery/sample_1/t0.1/vectors_corpus1
from LSCDiscovery.modules.utils_ import Space
import numpy
import csv
from sklearn.neighbors import NearestNeighbors 
#vecs_path = 'LSCDiscovery/output/en_semeval/BERT_layers1+12_typetoklem/discovery/sample_1/t0.1/vectors1'
#vecs_path = 'LSCDiscovery/output/en_semeval/BERT_layers1+12_typetoklem/discovery/sample_1/t0.1/vectors2'
vecs_path = 'NovelSenseDiscovery/data/data_files_vectors'
#output_file = 'predictions_coha.csv'
output_file = 'predictions_data_files.csv'
usage2sensepred = {}
nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto', metric='cosine').fit(sense_vecs)
for target in targets:
        
    path_matrix1 = vecs_path+'/'+target
    
    # Load matrices
    try: 
        try:
            space1 = Space(path_matrix1, format='npz') 
        except ValueError: 
            space1 = Space(path_matrix1, format='w2v') 
    except FileNotFoundError: 
        continue

    display(target)

    row2id1 = space1.row2id
    id2row1 = space1.id2row       
    #display(id2row1)
    vectors1 = space1.matrix.toarray() 
    #display(vectors1)
    #usage_vecs_no = vectors1.shape[0]
    #vecs = numpy.vstack((vectors1,sense_vecs))
    #vecs_no = vecs.shape[0]
    #row2id = {}
    #for (row,id) in row2id1 | row2id2 | row2id_sense:
    #    row2id[row] = row

    #Get nearest neighbor sense
    distances, indices = nbrs.kneighbors(vectors1) 
    #display(indices)
    for i, r in id2row1.items():
         usage2sensepred[target+'_'+str(i)] = id2row_senses[indices[i][0]] #todo add usage id

display(usage2sensepred)

predictions = [{'usage_id': u, 'sense_id':s} for (u,s) in usage2sensepred.items()]
# Export predictions
with open(output_file, 'w') as f:  
    w = csv.DictWriter(f, predictions[0].keys(), delimiter='\t', quoting = csv.QUOTE_NONE, quotechar='')
    w.writeheader()
    w.writerows(predictions)